# Постановка задачи

На сегодняшний день машинное обучение решает абсолютно разные задачи практически во всех областях нашей жизни. Платформа DataDriven организовывает контесты по машинному обучению для решения реальных задач общественных и некоммерческих организаций по всему миру, применяя результаты победителей. Данный проект призван решить проблемы водоснабжения в стране Танзания в Восточной Африке. Вода в Танзании добывается с помощью водокачек, и они часто выходят из строя, оставляя поселения без воды. Задача участников - по параметрам и статусу некоторого множества водокачек определить возможность поломки и необходимость обслуживания других водокачек. 

В данной задаче мы не только сделаем предсказания о состоянии водокачек, но и оценим, как качество данных влияет на предсказание.

Ссылка на контест: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/



# Описание метода

Задача будет решаться с помощью градиентного бустинга - будем использовать одноимённый метод GradientBoosting из пакета sklearn.

## Практическая суть задачи

Пусть у нас существует некоторый набор пар {(𝑥𝑖,𝑦𝑖)}, i=1..n, где x - это признак, y - целевая переменная. Задача сводится к тому, чтобы восстановить функцию y=F(x) - функцию, минимизирующую функцию потерь L(x, F(x))

## Градиентный бустинг

Метод градиентного бустинга позволяет построить итоговую модель F(x) как взвешенную функцию слабых моделей Fm(x), m = 1..M. К каждой предыдущей сумме Fm-1(x) добавляется частная производная от функции L(x, Fm(x)) по Fm(x) со знаком минус - для минимизации потерь. Назовём эту производную hm(x). Необходимо также, чтобы функция L(x, Fm(x)) была дифференцируема в достаточно большой области. 

In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier

Для работы будем использовать библиотеку Pandas. Выведем следующие данные:
TrainingX - множество признаков - набор данных о характеристиках водокачек
TrainingY - множество целевых переменных - набор данных о состоянии водокачек
TestX - множество признаков - набор данных о характеристиках водокачек, по которым требуется совершить предсказание.

In [2]:
TrainingX = pd.read_csv('./contest/trainingsetvalues.csv', sep=',')
TrainingY = pd.read_csv('./contest/trainingsetlabels.csv', sep=',')
TestX = pd.read_csv('./contest/testsetvalues.csv', sep=',')

Посмотрим, что представляют из себя данные о характеристиках водокачек:

In [3]:
print(type(TrainingX))

<class 'pandas.core.frame.DataFrame'>


In [4]:
TrainingX.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


# Пояснения к данным

Перечень столбцов с расшифровкой:
amount_tsh - Общий статический напор (количество воды, доступное для водокачки)
date_recorded - Дата ввода данных
funder - Кто финансировал колодец
gps_height - Глубина скважины
installer - Организация, вырывшая скважину
longitude - GPS-координата
latitude - GPS-координата
wpt_name - Название водоема, если он есть
num_private -
basin - Географическое расположение водоёма

subvillage - Географическое положение населённого пункта
region - Географическое положение региона
region_code - Код региона
district_code - Код района
lga - Географическое положение
ward - Географическое положение
population - Население вокруг скважины
public_meeting - Верно/Ложно
recorded_by - Групповой ввод этой строки данных
scheme_management - Кто управляет водокачкой
scheme_name - Кто управляет водокачкой
permit - Разрешен ли водозабор
construction_year - Год постройки водозабора
extraction_type - Тип добычи воды, которую использует водонапорная точка
extraction_type_group - Подвид добычи воды, которую использует водонапорная точка
extraction_type_class - Класс добычи воды, которую использует водонапорная точка
management - Как управляется водокачка
management_group - Как управляется водокачка
payment - Сколько стоит вода
payment_type - Сколько стоит вода
water_quality - Качество воды
quality_group - Качество воды
quantity - Количество воды
quantity_group - Количество воды
source - Источник воды
source_type - Источник воды
source_class - Источник воды
waterpoint_type - Вид водоема
waterpoint_type_group - Вид водоема


# Распределение данных по параметрам строковых типов
Чтобы включить в модель такие признаки, необходимо, чтобы было достаточно данных по каждому из них, и чтобы данные были достаточно равномерно распределены по ним. Для начала посмотрим, какие типы столбцов и в каком количестве у нас имеются:

In [5]:
TrainingX.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 40 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

Как мы видим,  строковых значений в таблице 30, целочисленных - 7, вещественных - 3. Посмотрим на таблицу, содержащую только строковые типы данных:

In [6]:
TrainingX.select_dtypes(include=['object'])

,date_recorded,funder,installer,wpt_name,basin,subvillage,region,lga,ward,public_meeting,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,2011-03-14,Roman,Roman,none,Lake Nyasa,Mnyusi B,Iringa,Ludewa,Mundindi,True,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,2013-03-06,Grumeti,GRUMETI,Zahanati,Lake Victoria,Nyamara,Mara,Serengeti,Natta,NaN,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,2013-02-25,Lottery Club,World vision,Kwa Mahundi,Pangani,Majengo,Manyara,Simanjiro,Ngorika,True,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,2013-01-28,Unicef,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mahakamani,Mtwara,Nanyumbu,Nanyumbu,True,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,2011-07-13,Action In A,Artisan,Shuleni,Lake Victoria,Kyanyamisa,Kagera,Karagwe,Nyakasimbi,True,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,2013-05-03,Germany Republi,CES,Area Three Namba 27,Pangani,Kiduruni,Kilimanjaro,Hai,Masama Magharibi,True,...,per bucket,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
59396,2011-05-07,Cefa-njombe,Cefa,Kwa Yahona Kuvala,Rufiji,Igumbilo,Iringa,Njombe,Ikondo,True,...,annually,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe
59397,2011-04-11,NaN,NaN,Mashine,Rufiji,Madungulu,Mbeya,Mbarali,Chimala,True,...,monthly,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
59398,2011-03-08,Malec,Musa,Mshoro,Rufiji,Mwinyi,Dodoma,Chamwino,Mvumi Makulu,True,...,never pay,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump


## Практическая оценка качества данных
Для начала попробуем оценить качество данных, построив модель напрямую. Оценим точность предсказания, разделив данные на две части: на одной обучим модель, по второй оценим точность.

Делим данные на два датафрейма:

In [57]:
TrainX1 = TrainingX[:29700]
TrainY1 = TrainingY[:29700]
TestX2 = TrainingX[29700:]
TestY2 = TrainingY[29700:]

Создаём объект модели:

In [58]:
model = GradientBoostingClassifier(
        n_estimators=500,
        max_depth=5,
        #subsample=0.5,
        max_features="auto"
        # random_state=seed
    )

Включаем в выборку все столбцы, приводимые к float напрямую, и приводим типы данных:

In [46]:
TrainX1.dtypes == object


id                       False
amount_tsh               False
date_recorded             True
funder                    True
gps_height               False
installer                 True
longitude                False
latitude                 False
wpt_name                  True
num_private              False
basin                     True
subvillage                True
region                    True
region_code              False
district_code            False
lga                       True
ward                      True
population               False
public_meeting            True
recorded_by               True
scheme_management         True
scheme_name               True
permit                    True
construction_year        False
extraction_type           True
extraction_type_group     True
extraction_type_class     True
management                True
management_group          True
payment                   True
payment_type              True
water_quality             True
quality_

Все столбцы, типом которых НЕ является object, мы можем привести к типу float. 

In [47]:
ctrainX = TrainX1[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
#ctrainX = ctrainX.values()
ctrainX = ctrainX.to_numpy()
ctrainX = ctrainX.astype(float)
ctrainX

array([[6.9572e+04, 6.0000e+03, 1.3900e+03, ..., 5.0000e+00, 1.0900e+02,
        1.9990e+03],
       [8.7760e+03, 0.0000e+00, 1.3990e+03, ..., 2.0000e+00, 2.8000e+02,
        2.0100e+03],
       [3.4310e+04, 2.5000e+01, 6.8600e+02, ..., 4.0000e+00, 2.5000e+02,
        2.0090e+03],
       ...,
       [7.2075e+04, 0.0000e+00, 1.1220e+03, ..., 2.0000e+00, 5.8000e+01,
        1.9730e+03],
       [6.0922e+04, 0.0000e+00, 0.0000e+00, ..., 4.0000e+00, 0.0000e+00,
        0.0000e+00],
       [5.6395e+04, 0.0000e+00, 0.0000e+00, ..., 1.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [48]:
TrainY1.replace(to_replace="functional", value=0, inplace=True)
TrainY1.replace(to_replace="non functional", value=1, inplace=True)
TrainY1.replace(to_replace="functional needs repair", value=2, inplace=True)

ctrainY = TrainY1
ctrainY = ctrainY.to_numpy()
ctrainY = ctrainY[:, 1]
ctrainY = ctrainY.astype(float)
ctrainY

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


array([0., 0., 0., ..., 1., 1., 1.])

In [49]:
ctestX = TestX2[['id',
 'amount_tsh',
 'gps_height',
 'longitude',
 'latitude',
 'num_private',
 'region_code',
 'district_code',
 'population',
 'construction_year'
                  ]]
ctestX = ctestX.to_numpy()
ctestX = ctestX.astype(float)


Строим модель:

In [50]:
model.fit(ctrainX,ctrainY)

#predictions = model.predict_proba(testX)[:,1]
predictions = model.predict_proba(ctestX)
predictions

array([[0.60938355, 0.35436435, 0.0362521 ],
       [0.15140095, 0.80165922, 0.04693983],
       [0.5603961 , 0.33886379, 0.1007401 ],
       ...,
       [0.54472616, 0.20820183, 0.24707201],
       [0.4426586 , 0.50347244, 0.05386896],
       [0.42246983, 0.54443401, 0.03309616]])

Мы получили таблицу вероятностей каждого из трех состояний. Выберем максимально вероятные состояния:

In [51]:
answ = predictions[:, 0].astype(int)
for ii in range (0, len(predictions)):
    answ[ii] = predictions[ii].argmax();
answ

array([0, 1, 0, ..., 0, 1, 1])

Теперь запишем их в датафрейм и сравним с действительными данными:

In [52]:
TestY = TestY2
TestY['status_group'] = answ
TestY

c:\python3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,status_group
29699,72779,0
29700,62962,1
29701,35938,0
29702,5107,0
29703,64811,0
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,1


In [53]:
TestY2

,id,status_group
29699,72779,0
29700,62962,1
29701,35938,0
29702,5107,0
29703,64811,0
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,1


In [54]:
TestY2.replace(to_replace="functional", value=0, inplace=True)
TestY2.replace(to_replace="non functional", value=1, inplace=True)
TestY2.replace(to_replace="functional needs repair", value=2, inplace=True)
TestY2

c:\python3\lib\site-packages\pandas\core\frame.py:5244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


,id,status_group
29699,72779,0
29700,62962,1
29701,35938,0
29702,5107,0
29703,64811,0
...,...,...
59395,60739,0
59396,27263,0
59397,37057,0
59398,31282,1


In [55]:
(TestY2['status_group'] == TestY['status_group']).sum()

29701

In [56]:
wrk = 0
dntwrk = 0
ndrpr = 0

answ = predictions[:,0].astype(int)



## Оптимизация данных
Для начала нам надо определить, есть ли записи об одних и тех же водокачках и насколько различаются данные по ним. Уникальными значениями для каждой из водокачек являются их GPS-координаты. Оценим, как много в таблице уникальных водокачек по координатам:

In [ ]:
len(TrainingX['latitude'].unique())

In [ ]:
len(TrainingX['longitude'].unique())

In [ ]:
Train = TrainingX.loc[TrainingX['longitude'] != 0]
#TrainingX.groupby('latitude')['longitude']

Train[Train.duplicated(subset='longitude',keep=False)][Train.duplicated(subset='latitude',keep=False)]




Как мы видим, нашлось 138 водокачек с одинаковыми ненулевыми координатами. Многие из них были профинансированы или построены одними и теми же лицами, что может указывать на то, что это данные об одних и тех же водокачках. Проверим их на совпадение годов ввода в эксплуатацию:

In [ ]:
TrainYEAR = Train[Train.duplicated(subset='longitude',keep=False)][Train.duplicated(subset='latitude',keep=False)]
TrainYEAR[TrainYEAR.duplicated(subset='construction_year', keep = False)]

Посмотрим на повторяющиеся данные:

In [ ]:
TrainYEAR.loc[abs(TrainYEAR['longitude'] - 37.3) < 0.1]

26 строк с повторяющимися данными. Отсортируем по возрастанию долготы:

In [ ]:
TrainYEARSORT = TrainYEAR.loc[abs(TrainYEAR['longitude'] - 37.3) < 0.1]
TrainYEARSORT.sort_values(by = 'longitude')

In [ ]:
Видим следующее: 
    1. У каждой из дублирующихся водокачек есть по 2 записи
    2. Даты первых и вторых записей у большинства из них совпадают
    3. Даты записей всех дублирующихся водокачек близки друг к другу
    
Делаем предположения:
    1. Сбором данных занимались две несообщённые группы - в таком случае можно исключить данные из выборки при условии
        неверности нижеследующего пункта.
    2. Записи сделаны по причине изменения состояния водокачек - проверяем.

In [ ]:
Извлечём из данных о состоянии водокачек две случайные записи под номерами, 
под которыми они дублируются в данных об их характеристиках:

In [ ]:
#NewTrain = pd.merge(TrainYEARSORT, TrainingY, on = 'id','status_group', how = 'right')
#NewTrain

val = TrainYEARSORT
val1 = TrainingY
val1 = val1.merge(val, on = 'id', how = 'right')
val1.sort_values(by = 'latitude')

Случайная проверка показала, что утверждение №2 неверно. Делаем вывод, что дублирующиеся данные скорее всего не дают никакой полезной информации, поэтому их можн (и даже нужно) удалить. Однако есть проблема: не во всех строчках заполнены все поля. Нам необходимо либо их дозаполнить, либо удалить записи по тем столбцам, которые заполнены у всех. 

## Дата ввода данных
Оценим разброс дат ввода данных. Если ввод был совершён слишком давно или слишком недавно относительно большинства дат, то такая дата может негативно сказаться на точности прогноза, т.к. если, например, запись была совершена за 40 